## 1. 코랩 환경 설정

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
!pip install transformers
!pip install datasets
!git clone https://github.com/ZIZUN/korean-malicious-comments-dataset.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

1. Colab 환경 설정 및 GPU 출력

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import load_dataset
import numpy as np


In [5]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ",device)

device =  cuda


## 2. 데이터셋 제작

### 1. 데이터 load 및 EDA

In [6]:
df_kmc = pd.read_csv("/content/korean-malicious-comments-dataset/Dataset.csv",sep="\t")
splits = {'train': 'beep.train.csv', 'test': 'apeach.test.csv'}
df_apeach = pd.read_csv("hf://datasets/jason9693/APEACH/" + splits["train"])
df_apeach_test = pd.read_csv("hf://datasets/jason9693/APEACH/" + splits["test"])

# 출력
print("korean-malicious-comments datasets")
display(df_kmc.head())

print("apeach datasets")
display(df_apeach.head())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


korean-malicious-comments datasets


,content,lable
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,0.0
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,0.0
2,짱깨 꺼라ㅡ패쓰,0.0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,1.0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1.0


apeach datasets


,text,class
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,1
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",1
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,1


In [7]:
print("============kmc datasets info============")
display(df_kmc.info())
print("============apeach datasets info============")
display(df_apeach.info())

============kmc datasets info============
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   content  10000 non-null  object 
 1   lable    9975 non-null   float64
dtypes: float64(1), object(1)
memory usage: 156.4+ KB


None

============apeach datasets info============
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7896 entries, 0 to 7895
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7896 non-null   object
 1   class   7896 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 123.5+ KB


None

- kmc 데이터셋은 10,000개의 데이터가 정확하게 load 되었지만, 악성 댓글의 여부를 나타내는 Label은 25개의 null 데이터가 존재하는걸 확인

- apeach 데이터셋은 7896의 train 데이터셋으로 확인

In [8]:
null_idx = df_kmc[df_kmc['lable'].isnull()].index
df_kmc.loc[null_idx,"content"]

1602    응애 응애 엄마 저 맘에 안들죠? ........아들 ?? " 너 내가 우스워 보이...
1654           토니스타크 평소 "아이엠그루트"라는 유행어를 부러워했다는게 학계의 정설\t1
1992    "13일 현대차에 따르면 올 들어 국내 소비자들의 수입차 구매의향률이 3년 만에 하...
2920                 에이프릴이 한마디 합니다 "예쁜게 죄" 구하라님 "무기징역"\t1
3720          답글 글씨체를 봐라 저게 애새끼가 쓴거냐?"빨갱이새끼가 쓴거지 ㅁㅈㅎㅉㅉ\t0
3807    알겠다이기ㅋㅋ 딱 채찍쳐맞는거 좋아하는 한국식 마인드네. 노예마인드. 조금만 성공한...
3908           이래서 스스로 걸리거든 "죄인들이"~ㅎㅎㅎ 재미보고 털리고 그치~~~?\t0
4241    아버지는 내재된 악마들을 다룰 정신적 힘을 가지고 있지 않았다." 이 말한마디가 사...
4283    댓글 중 "선동 당해서 촞불든 개돼지 홍어들도 단죄를 받아야 할 공범자들이다"에10...
5000    스파이 제안받고 살해 안당하는 법1. 처음에 스파이 제안을 받았을때 "중국을 위해서...
5521    "국방부 "까지 ㅡㄱ ㅐ 엿같은 ㅈ ㅣ랄주댕이...좌빨에서 ㅡ인민군대로 ㅡ가려는건가...
5866    쌩뚱맞게 60대최반엌 치매라니 그것도 곱게 사는 사모님이- -" 알콜중독도 아니고 ...
6477    페미메퇘지쿵쾅년인 메갈페미들은 니들이 좋아하는 싫어요 ㄱㄱ제발부탁해~~"일반 여성"...
6538    아니 ㅆㅂ 그런 "카더라"가 넘쳐난다고 그거에 대해서 혹시 댓글게이는 뭔가 아는거 ...
6771    저 때 투니버스에서 코요태 짧게 인터뷰 했었는데 김종민이 "노래는 뭐 신지가 다 하...
6932               개 족 가튼 국방부의 "휴기연장콜센터"발족을 축하한다 ㅆ ㅂ..\t0
7199    민족적 자존심과 애국심을 갖고 국산품 이용합시다 . . . "겸손"한 마음으로 재산...
7252    아나운서는 

데이터 Load 할때 "\t" 를 sep 파라미터로 설정했는데, "\t" 앞에 특수 문자가 있으면 데이터가 정확하게 분리 되지 않음을  확인함. => 추가 전처리 작업 필요

### 2. 데이터 전처리

#### 1. df_kmc 데이터 전처리

In [9]:
# lable -> label로 colums 열이름 수정
df_kmc.rename(columns={"lable":"label"},inplace=True)

#label은 content의 가장 끝 문자열로 지정
df_kmc.loc[null_idx,"label"] = df_kmc.loc[null_idx,"content"].str[-1]

# content는 "\t" 앞부분까지의 문자열로 설정
df_kmc.loc[null_idx,"content"] = df_kmc.loc[null_idx,"content"].str[:-2]

In [10]:
# 학습을 위해 label의 데이터 타입 int로 변환
df_kmc = df_kmc.astype({"label": int})
df_kmc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  10000 non-null  object
 1   label    10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


### 3. Train set /Test set으로 나누기

In [11]:
# KMC 데이터셋
df_kmc_train = df_kmc.sample(frac=0.8,random_state=42)
df_kmc_test = df_kmc.drop(df_kmc_train.index)

# Apeach 데이터셋
df_apeach_train = df_apeach.copy()

print("====================KMC 데이터셋====================")
display("df_kmc train set")
display(df_kmc_train.info())
display("df_kmc test set")
display(df_kmc_test.info())

print("====================Apeach 데이터셋====================")
display("df_apeach train set")
display(df_apeach_train.info())
display("df_apeach test set")
display(df_apeach_test.info())


====================KMC 데이터셋====================


'df_kmc train set'

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 6252 to 6907
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  8000 non-null   object
 1   label    8000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 187.5+ KB


None

'df_kmc test set'

...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7896 entries, 0 to 7895
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7896 non-null   object
 1   class   7896 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 123.5+ KB


None

'df_apeach test set'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3770 entries, 0 to 3769
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3770 non-null   object
 1   class   3770 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 59.0+ KB


None

#### 데이터 피처 이름 통일
- text -> content
- class -> label

In [12]:
# 데이터 병합 전 데이터 columns 이름 변경(통일) : text->content, class->label
df_apeach_train.rename(columns={"text":"content","class":"label"},inplace=True)
df_apeach_test.rename(columns={"text":"content","class":"label"},inplace=True)

df_apeach_train.head()

,content,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,1
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",1
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,1


#### 데이터 병합

In [20]:
# kmc, apeach train 데이터 병합
train_data = pd.concat([df_kmc_train,df_apeach_train],axis=0)

# kmc, apeach test 데이터 병합
test_data = pd.concat([df_kmc_test,df_apeach_test],axis=0)

# 데이터 info
print("====================Train 데이터셋====================")
display(train_data.info())
print("====================Test 데이터셋====================")
display(test_data.info())

====================Train 데이터셋====================
<class 'pandas.core.frame.DataFrame'>
Index: 15896 entries, 6252 to 7895
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  15896 non-null  object
 1   label    15896 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 372.6+ KB


None

====================Test 데이터셋====================
<class 'pandas.core.frame.DataFrame'>
Index: 5770 entries, 1 to 3769
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  5770 non-null   object
 1   label    5770 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 135.2+ KB


None

#### 데이터 중복 제거

In [21]:
# 데이터셋 갯수 확인
print("중복 제거 전 학습 데이터셋 : {}".format(len(train_data)))
print("중복 제거 전 테스트 데이터셋 : {}".format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=["content"],inplace=True)
test_data.drop_duplicates(subset=["content"],inplace=True)

# 데이터셋 갯수 확인
print("중복 제거 후 학습 데이터셋 : {}".format(len(train_data)))
print("중복 제거 후 테스트 데이터셋 : {}".format(len(test_data)))

중복 제거 전 학습 데이터셋 : 15896
중복 제거 전 테스트 데이터셋 : 5770
중복 제거 후 학습 데이터셋 : 12033
중복 제거 후 테스트 데이터셋 : 5770


중복 제거 후 학습 데이터셋의 개수가 줄어들었음.

### 4. 토크나이징

#### 1. 토크나이저  불러오기

In [69]:
model_name = "beomi/KcELECTRA-small-v2022"
#model_name = "beomi/KcELECTRA-base"  #-> 토크나이저 오류

tokenizer = AutoTokenizer.from_pretrained(model_name)

다음으로 텍스트 형태의 데이터를 모델이 학습할 수 있는 토큰 형태로 분리하고 이를 토큰 id 값으로 변환한다.

이는 transformers 라이브러리의 AutoTokenizer 모듈을 활용해 손쉽게 진행할 수 있다.


huggingface의 model 로 등록된 모델의 경우 모델뿐만 아니라 학습에 사용된 tokenizer까지 업로드되어 있다. AutoTokenizer는 이를 활용하는 모듈로 사용하고자 하는 모델명만 입력하면 자동으로 해당 토크나 이저를 불러올 수 있다.

#### 2. 데이터를 토크나이징 하기

#### train 데이터셋 토크나이징

In [83]:
tokenized_train_sentences = tokenizer(
    list(train_data["content"]),
    return_tensors="pt",      # PyTorch의 tensor 형태로 반환
    max_length=128,           # 최대 토큰 길이
    padding=True,             # 짧은 문장은 제로 패딩을 추가
    truncation=True,          # 최대 길이를 초과하는 문장은 잘라냄
    add_special_tokens=True,  # 특별 토큰 추가
)

In [84]:
# 첫 번째 문장 출력
print("<첫번째 문장 정보>")
print(tokenized_train_sentences[0])
print()

print("<tokenized train 문장>")
print(tokenized_train_sentences["input_ids"][0])
print()

print("<Decoded Sentence>")
decoded_sentence = tokenizer.decode(tokenized_train_sentences["input_ids"][0], skip_special_tokens=True)
print(decoded_sentence)
print()

print("<Tokenized Train 문장의 토큰>")
print(tokenized_train_sentences[0].tokens)
print()

print("<Tokenized Train 문장의 토큰 ID>")
print(tokenized_train_sentences["input_ids"][0].tolist())
print()


print("<Tokenized Train 문장의 Attention Mask>")
print(tokenized_train_sentences["attention_mask"][0].tolist())

<첫번째 문장 정보>
Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

<tokenized train 문장>
tensor([    2, 17047,    96,    96, 26515, 11692, 16559,  4229,  3456, 24893,
         4079, 20748, 14172, 15643,    96,  8437, 13221,  9435,  4059, 13654,
         9167, 15127,  8348, 15643,    96,    96,     3,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,   

반환된 데이터를 보면 ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing 값들을 포함하 고 있는 것을 확인할 수 있으며, tokens 값에는 [CLS], [SEP], [PAD] 과 같은 special_token을 포함하여 텍스트가 분리된 것을 확 인할 수 있다.

또한 ids 값을 통해 각 token의 id를 확인할 수 있으며, attention_mask 값을 통해 패딩된 값을 구분할 수 있다.

#### test 데이터셋 토크나이닝

In [85]:
tokenized_test_sentences = tokenizer(
    list(test_data["content"]),
    return_tensors="pt",      # PyTorch의 tensor 형태로 반환
    max_length=128,           # 최대 토큰 길이
    padding=True,             # 짧은 문장은 제로 패딩을 추가
    truncation=True,          # 최대 길이를 초과하는 문장은 잘라냄
    add_special_tokens=True,  # 특별 토큰 추가
)

### 5. 데이터셋 생성

pytorch 프레임워크에서 모델이 학습할수 있는 형태의 데이터셋 생성

학습을 위한 데이터셋 클래스 만들기

In [86]:
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

이후 train_set, test_set에 대한 데이터셋 각각 생성

In [87]:
train_label = train_data["label"].values
test_label = test_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [88]:
#데이터셋 드라이브에 다운
torch.save(train_dataset, '/gdrive/MyDrive/Colab Notebooks/train_dataset.pt')
torch.save(test_dataset, '/gdrive/MyDrive/Colab Notebooks/test_dataset.pt')

## 모델 학습

#### 1. 모델 불러오기
사전 학습된 KcELECTRA 모델 불러옴.

욕설 분류 task를 위한 모델이니까 AutoModelForSequenceClassification

In [89]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

pytorch_model.bin:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-small-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Li

#### 2. 학습 파라미터 설정